In [1]:
import $ivy.`org.apache.spark:spark-sql_2.13:3.3.2`
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

val spark = SparkSession.builder()
  .appName("SparkTest")
  .master("local[*]")
  .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
  
// http://localhost:4040/jobs/
println("Spark is ready!")

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties


Spark is ready!


import $ivy.$                                      

import org.apache.spark.sql._

import org.apache.spark.sql.functions._


spark: SparkSession = org.apache.spark.sql.SparkSession@25bef652

In [2]:
import spark.implicits._
import java.sql.Date

case class Activity(
  player_id: Int,
  device_id: Int,
  event_date: Date,
  games_played: Int
)

val activityData: Seq[(Int, Int, Date, Int)] = Seq(
  (1, 2, Date.valueOf("2016-03-01"), 5),
  (1, 2, Date.valueOf("2016-03-02"), 6),
  (2, 3, Date.valueOf("2017-06-25"), 1),
  (3, 1, Date.valueOf("2016-03-02"), 0),
  (3, 4, Date.valueOf("2018-07-03"), 5)
)

val activityDS = activityData.toDF(
  "player_id",
  "device_id",
  "event_date",
  "games_played"
).as[Activity]

import spark.implicits._

import java.sql.Date


defined class Activity
activityData: Seq[(Int, Int, Date, Int)] = List(
  (1, 2, 2016-03-01, 5),
  (1, 2, 2016-03-02, 6),
  (2, 3, 2017-06-25, 1),
  (3, 1, 2016-03-02, 0),
  (3, 4, 2018-07-03, 5)
)
activityDS: Dataset[Activity] = [player_id: int, device_id: int ... 2 more fields]

Write a solution to report the fraction of players that logged in again on the day after the day they first logged in, rounded to 2 decimal places. In other words, you need to count the number of players that logged in for at least two consecutive days starting from their first login date, then divide that number by the total number of players.

The result format is in the following example.
| fraction  |
|-----------|
| 0.33      |

In [3]:
activityDS.show()

val playerCount = activityDS.select(col("player_id"))
    .dropDuplicates().count()

+---------+---------+----------+------------+
|player_id|device_id|event_date|games_played|
+---------+---------+----------+------------+
|        1|        2|2016-03-01|           5|
|        1|        2|2016-03-02|           6|
|        2|        3|2017-06-25|           1|
|        3|        1|2016-03-02|           0|
|        3|        4|2018-07-03|           5|
+---------+---------+----------+------------+



playerCount: Long = 3L

In [ ]:
activityDS.as("a").groupBy(col("player_id"))
    .agg(min("event_date").as("event_date_first"))
    .join(
        activityDS.as("b"), 
        (col("a.player_id") === col("b.player_id")) &&
        (date_add(col("event_date_first"), 1) === col("event_date"))
    )
    .groupBy(col("a.player_id"))
    .agg(round(count(lit(1))/playerCount, 2).as("fraction"))
    .select(col("fraction"))
    .show()

(console):7:5 expected end-of-input
    .groupBy(col("a.player_id"))
    ^

: 

In [ ]:
spark.stop